In [1]:
import sys
sys.path.append('/media/arclabdl1/HD1/Linjun/mpc-mpnet-py/deps/sparse_rrt-1')
from sparse_rrt import _sst_module
from sparse_rrt.systems import standard_cpp_systems

import numpy as np
import time

In [2]:
# from sparse_rrt.experiments.experiment_utils import run_config
# point_config = dict(
#     system='point',
#     planner='sst_backend',
#     start_state=[9., 9.],
#     goal_state=[-9., 9.],
#     goal_radius=0.5,
#     random_seed=100,
#     integration_step=0.02,
#     min_time_steps=2,
#     max_time_steps=20,
#     number_of_iterations=300000,
#     display_type=None,
#     sst_delta_near=0.2,
#     sst_delta_drain=0.2
# )
# run_config(point_config)


def test_sst_acrobot(min_time_steps, max_time_steps, integration_step, number_of_iterations):
    '''
    Test that runs are deterministic
    '''
    system = standard_cpp_systems.TwoLinkAcrobot()

    def _create_planner(system):
        return _sst_module.SSTBackendWrapper(
            state_bounds=system.get_state_bounds(),
            control_bounds=system.get_control_bounds(),
            distance=system.distance_computer(),
            start_state=np.array([0., 0., 0, 0,]),
            goal_state=np.array([3, 0, 0, 0]),
            goal_radius=10,
            random_seed=0,
            sst_delta_near=0.2,
            sst_delta_drain=0.2
        )
    planner = _create_planner(system)
    start_time = time.time()

    for iteration in range(number_of_iterations):
        planner.step(system, min_time_steps, max_time_steps, integration_step)
        solution = planner.get_solution()
        if iteration % 1000 == 0:
            if solution is None:
                solution_cost = None
            else:
                solution_cost = np.sum(solution[2])

            print("Time: %.2fs, Iterations: %d, Nodes: %d, Solution Quality: %s" %
                  (time.time() - start_time, iteration, planner.get_number_of_nodes(), solution_cost))
    return planner

p = test_sst_acrobot(1, 100, 2e-2, 2000)

Time: 0.00s, Iterations: 0, Nodes: 2, Solution Quality: None
Time: 0.52s, Iterations: 1000, Nodes: 917, Solution Quality: 14.700000000000003


In [3]:
p.nearest_vertex(np.array([0, 0, 0, 0]))

array([0., 0., 0., 0.])

In [4]:
p.add_to_tree(np.array([3, 0, 0, 0]), 6.2)

In [5]:
p.get_solution()

(array([[0., 0., 0., 0.],
        [3., 0., 0., 0.]]),
 array([[4.68628294e-310]]),
 array([6.2]))

In [6]:
import sys
sys.path.append('/media/arclabdl1/HD1/Linjun/mpc-mpnet-py/deps/sparse_rrt-1')
from sparse_rrt import _sst_module
from sparse_rrt.systems import standard_cpp_systems

import pickle
from mpnet.sst_envs.utils import load_data


model = "acrobot_obs"
env_id = 1
traj_id = 10
filepath = "/media/arclabdl1/HD1/Linjun/data/kinodynamic/{model}/{filetype}_{env_id}.pkl".format(model=model, env_id=env_id, filetype="obs")

data = load_data(model, env_id, traj_id)
path = data['path']
obs_list = pickle.load(open(filepath, "rb")).reshape(-1, 2)
width = 6

env_vox = np.load("mpnet/sst_envs/acrobot_obs_env_vox.npy")
obc = env_vox[env_id,0]
p = _sst_module.DSSTMPCWrapper(
            start_state=np.array(path[0]),
            goal_state=np.array(path[-1]),
            goal_radius=2,
            random_seed=0,
            sst_delta_near=0.2,
            sst_delta_drain=0.2,
            obs_list=obs_list,
            width=width,
            verbose=True
        )

In [ ]:
p.step(1, 10, 2e-2)

In [8]:
def test_dsst_acrobot(min_time_steps, max_time_steps, integration_step, number_of_iterations):
    '''
    Test that runs are deterministic
    '''
#     system = standard_cpp_systems.TwoLinkAcrobot()

    def _create_planner():
        return _sst_module.DSSTMPCWrapper(
            start_state=np.array(path[0]),
            goal_state=np.array(path[-1]),
            goal_radius=2,
            random_seed=0,
            sst_delta_near=0.2,
            sst_delta_drain=0.2,
            obs_list=obs_list,
            width=width,
            verbose=True
        )
    planner = _create_planner()
    return planner
    start_time = time.time()

    for iteration in range(number_of_iterations):
        planner.step(min_time_steps, max_time_steps, integration_step)
#         planner.neural_step(obc.reshape(-1))
        solution = planner.get_solution()
        if iteration % 1000 == 0:
            if solution is None:
                solution_cost = None
            else:
                solution_cost = np.sum(solution[2])

            print("Time: %.2fs, Iterations: %d, Nodes: %d, Solution Quality: %s" %
                  (time.time() - start_time, iteration, planner.get_number_of_nodes(), solution_cost))
    return planner

p = test_dsst_acrobot(1, 100, 2e-2, 100000)

In [9]:
p.get_solution()

In [10]:
np.sum(data['cost'])

4.78

In [11]:
path

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.24618453, -0.5358435 ,  1.08646313, -2.47822479],
       [ 0.40320096, -0.92563333,  0.55125302, -1.6938076 ],
       [-0.44048159,  2.43457069,  2.77709511, -0.25851318],
       [-0.08082761,  2.34253993,  3.19968799, -1.41360991],
       [ 0.29280925,  2.15194762,  2.89925181, -1.74592252],
       [-1.00135378, -2.4274398 , -2.32588713,  3.06098477],
       [ 1.98763322,  1.0434565 , -0.18161975, -1.57335805]])

In [ ]:
p.step(1, 100, 2e-2)